# LangChain Async Example

This notebook demonstrates how to use the streaming capability of langchain and monitor the results using trulens. Example app by user Anunaya Joshi .

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.parent.parent.resolve()))

In [ ]:
import asyncio

from IPython.display import display
from ipywidgets import interact
from ipywidgets import widgets
from langchain import LLMChain
from langchain import PromptTemplate
from langchain.callbacks import AsyncIteratorCallbackHandler
from langchain.chains import LLMChain
from langchain.chat_models.openai import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.memory import ConversationSummaryBufferMemory

from trulens_eval import Feedback
from trulens_eval import feedback
from trulens_eval import Tru
from trulens_eval.keys import check_keys
import trulens_eval.utils.python  # makes sure asyncio gets instrumented

Tru().reset_database()

check_keys(
    "OPENAI_API_KEY",
    "HUGGINGFACE_API_KEY"
)

In [ ]:
# Set up a language match feedback function.

tru = Tru()
hugs = feedback.Huggingface()
f_lang_match = Feedback(hugs.language_match).on_input_output()

# Set up an async callback.
callback = AsyncIteratorCallbackHandler()

chatllm = ChatOpenAI(
    temperature=0.0,
    streaming=True# important
    # callbacks=[callback]
)
llm = OpenAI(
    temperature=0.0,
)

memory = ConversationSummaryBufferMemory(
    memory_key="chat_history",
    input_key="human_input",
    llm=llm,
    max_token_limit=50
)

# Setup a simple question/answer chain with streaming ChatOpenAI.
prompt = PromptTemplate(
    input_variables=["human_input", "chat_history"],
    template='''
    You are having a conversation with a person. Make small talk.
    {chat_history}
        Human: {human_input}
        AI:'''
)

chain = LLMChain(llm=chatllm, prompt=prompt, memory=memory)
tc = tru.Chain(chain, feedbacks=[f_lang_match], app_id="chat_with_memory")

message = "Hi. How are you?"

# Create a task with the call to the chain, but don't wait for it yet.
f_res_record = asyncio.create_task(
    tc.acall_with_record(
        inputs=dict(human_input=message),
        callbacks=[callback]
    )
)

# Instead wait for the callback's async generator, getting us each token as it comes in.
async for token in callback.aiter():
    print(token)

# By now the acall_with_record results should be ready.
res, record = await f_res_record

In [ ]:
# Inspect the result:

res

In [ ]:
# Inspect the record:

record.dict()

In [ ]:
# Interactive version of the above.


out = widgets.HTML(value="", layout=dict(width="50%"))
mem_out = widgets.HTML(value="", layout=dict(width="50%"))
display(widgets.HBox([out, mem_out]))

chatllm = ChatOpenAI(
    temperature=0.0,
    streaming=True# important
    # callbacks=[callback]
)
llm = OpenAI(
    temperature=0.0,
)

memory = ConversationSummaryBufferMemory(
    memory_key="chat_history",
    input_key="human_input",
    llm=llm,
    max_token_limit=50
)

# Setup a simple question/answer chain with streaming ChatOpenAI.
prompt = PromptTemplate(
    input_variables=["human_input", "chat_history"],
    template='''
    You are having a conversation with a person. Make small talk.
    {chat_history}
        Human: {human_input}
        AI:'''
)

chain = LLMChain(llm=chatllm, prompt=prompt, memory=memory)
tc = tru.Chain(chain, feedbacks=[f_lang_match], app_id="chat_with_memory")

async def process_question(message):
    if message == "":
        return

    out.value += "<b>Human:</b> " + message + "<br/><br/><b>AI:</b> "
    
    # Set up an async callback.
    callback = AsyncIteratorCallbackHandler()

    f_res_record = asyncio.create_task(
        tc.acall_with_record(
            inputs=dict(human_input=message),
            callbacks=[callback]
        )
    )
    
    # Instead wait for the callback's async generator, getting us each token as it comes in.
    async for token in callback.aiter():
        out.value += token

    out.value += "<br/><br/>"

    # By now the acall_with_record results should be ready.
    res, record = await f_res_record

    temp = "<b>Memory summary:</b><br/>" + memory.moving_summary_buffer + "<br/><br/><b>Memory buffer</b>:</br>"
    for msg in memory.chat_memory.messages:
        temp += str(msg.content) + "<br/><br/>"
    mem_out.value = temp

@interact(message=widgets.Text(continuous_update=False))
def ask_question(message: str):
    asyncio.ensure_future(process_question(message))

In [ ]:
# Start the dashboard. `_dev` is for running from the github repo:

Tru().start_dashboard(force=True, _dev=Path.cwd().parent.parent.parent)